This notebook demonstrates how to use the IBM Quantum Algorithm library to invoke a specific algorithm and process the result.

Further information is available for the algorithms in the github repo algorithms/readme.md

In [1]:
import paths
from qiskit_acqua import Operator, run_algorithm, get_algorithm_instance
from qiskit_acqua.input import get_input_instance
from qiskit_acqua.ising import maxcut
import numpy as np
import random

Here an Operator instance is created for our Hamiltonian. In this case the paulis are from an Ising Hamiltonian of the max cut problem. We load a small instance of the maxcut problem.

In [2]:
qubitOp, offset, w, n = maxcut.parse_gset_format('sample.maxcut')
algo_input = get_input_instance('EnergyInput')
algo_input.qubit_op = qubitOp

We also offer a function to generate a random graph as a input.

In [3]:
if False:
    qubitOp, offset, w, n = maxcut.random_maxcut(10, savefile='sample2.maxcut', seed=123)
    algo_input.qubit_op = qubitOp

In [4]:
to_be_tested_algos = ['ExactEigensolver', 'CPLEX', 'VQE']
operational_algos = []
for algo in to_be_tested_algos:
    try:
        get_algorithm_instance(algo)
        operational_algos.append(algo)
    except:
        print("{} is unavailable, please check your setting.".format(algo))
print(operational_algos)

['ExactEigensolver', 'CPLEX', 'VQE']


We can now use the Operator without regard to how it was created. First we need to prepare the configuration params to invoke the algorithm. Here we will use the ExactEigensolver first to return the smallest eigenvalue. Backend is not required since this is computed classically not using quantum computation. We then add in the qubitOp Operator in dictionary format. Now the complete params can be passed to the algorithm and run. The result is a dictionary.

In [5]:
if 'ExactEigensolver' not in operational_algos:
    print("ExactEigensolver is not in operational algorithms.")
else:
    algorithm_cfg = {
        'name': 'ExactEigensolver',
    }

    params = {
        'problem': {'name': 'ising'},
        'algorithm': algorithm_cfg
    }
    result = run_algorithm(params,algo_input)
    # print('objective function:', maxcut.maxcut_obj(result, offset))
    x = maxcut.convert_eigevecs(n, result['eigvecs'])
    print('energy:', result['energy'])
    print('maxcut value:', maxcut.maxcut_value(x, w))
    print('solution:', maxcut.convert_gset_result(x))

energy: -10.0
maxcut value: 15.0
solution: {1: 1, 2: 0, 3: 1, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 0}


We change the configuration parameters to solve it with the CPLEX backend. The CPLEX backend can deal with a particular type of Hamiltonian called Ising Hamiltonian, which consists of only Pauli Z at most second order and often appears for combinatorial optimization problems such as the max-cut problem.

In [6]:
if 'CPLEX' not in operational_algos:
    print("CPLEX is not in operational algorithms.")
else:
    algorithm_cfg = {
        'name': 'CPLEX',
        'display': 0
    }

    params = {
        'problem': {'name': 'ising'},
        'algorithm': algorithm_cfg
    }

    result = run_algorithm(params, algo_input)

    #print('objective function:', maxcut.maxcut_obj(result, offset))
    x = result['x_sol']
    print('energy:', result['energy'])
    print('maxcut value:', maxcut.maxcut_value(x, w))
    print('solution:', maxcut.convert_gset_result(x))
    print('time:', result['time'])

CPXPARAM_TimeLimit                               600
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Tolerances_Integrality              0
CPXPARAM_MIP_Display                             0
energy: -10.0
maxcut value: 15.0
solution: {1: 0, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 1, 10: 1}
time: 0.013403612014371902


Now we want VQE and so change it and add its other configuration parameters. VQE also needs and optimizer and variational form. While we can omit them from the dictionary, such that defaults are used, here we specify them explicitly so we can set their parameters as we desire.

In [7]:
if 'VQE' not in operational_algos:
    print("VQE is not in operational algorithms.")
else:
    algorithm_cfg = {
        'name': 'VQE',
        'operator_mode': 'matrix'
    }

    optimizer_cfg = {
        'name': 'L_BFGS_B',
        'maxfun': 1000
    }

    var_form_cfg = {
        'name': 'RYRZ',
        'depth': 7,
        'entanglement': 'linear'
    }

    params = {
        'problem': {'name': 'ising'},
        'algorithm': algorithm_cfg,
        'optimizer': optimizer_cfg,
        'variational_form': var_form_cfg,
        'backend': {'name': 'local_statevector_simulator'}
    }

    result = run_algorithm(params,algo_input)
    #print('objective function:', maxcut.maxcut_obj(result, offset))
    x = maxcut.convert_eigevecs(n, np.abs(result['eigvecs']))
    print('energy:', result['energy'])
    print('maxcut value:', maxcut.maxcut_value(x, w))
    print('solution:', maxcut.convert_gset_result(x))
    print('time:', result['eval_time'])

energy: -4.678943465525099
maxcut value: 15.0
solution: {1: 0, 2: 1, 3: 0, 4: 0, 5: 1, 6: 1, 7: 0, 8: 0, 9: 1, 10: 1}
time: 231.5777349472046


We solve an instance of Gset benchmark dataset (https://web.stanford.edu/~yyye/yyye/Gset/ ). Because the graph size is too large for ExactEigensolver and VQE, we apply it to CPLEX backend. Note that you can find the best known values of Gset at http://www.optsicom.es/maxcut/maxcut_bestvalues.xls. For example, the best known solution of G11 is 564 and it matches the result by CPLEX backend.

In [8]:
if 'CPLEX' not in operational_algos:
    print("CPLEX is not in operational algorithms.")
else:
    qubitOp, offset, w, n = maxcut.parse_gset_format('G11')
    algo_input = get_input_instance('EnergyInput')
    algo_input.qubit_op = qubitOp

    algorithm_cfg = {
        'name': 'CPLEX',
        'timelimit': 600,
        'thread': 2
    }

    params = {
        'problem': {'name': 'ising'},
        'algorithm': algorithm_cfg
    }

    result = run_algorithm(params, algo_input)
    print('objective function:', maxcut.maxcut_obj(result, offset))
    x = result['x_sol']
    print('energy:', result['energy'])
    print('maxcut value:', maxcut.maxcut_value(x, w))
    #print('solution:', maxcut.convert_gset_result(x))
    print('time:', result['time'])

CPXPARAM_TimeLimit                               600
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 2
CPXPARAM_MIP_Tolerances_Integrality              0
Found incumbent of value 34.000000 after 0.00 sec. (0.15 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 1 columns.
MIP Presolve added 3200 rows and 1600 columns.
Reduced MIP has 3200 rows, 2400 columns, and 6400 nonzeros.
Reduced MIP has 2400 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.62 ticks)
Probing time = 0.00 sec. (1.05 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 817 rows and 0 columns.
Reduced MIP has 2383 rows, 2400 columns, and 5583 nonzeros.
Reduced MIP has 2400 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (5.05 ticks)
Probing time = 0.00 sec. (1.00 ticks)
Clique table members: 1566.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: